# 0. Initialisering
For å komme i gang må vi importere relevante biblioteker og en gyldig openai-nøkkel.
For enkelhets skyld definerer vi også en funksjon for å forenkle kall av OpenAI ChatCompletion API'et.

In [ ]:
# OpenAI initialisation and additional dependencies
import os
import openai
from pprint import pprint
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
# Helper function for calling ChatCompletion API

# Alternative models to try: gpt-4-1106-preview, gpt-4, gpt-3.5-turbo-0125, gpt-3.5-turbo, gpt-3.5-turbo-0613

def get_completion(prompt, model="gpt-3.5-turbo-0613"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# 1. Grunnleggende prompt engineering-prinsipper, med noen eksempler

## 1.1 Prinsipp 1: Skriv klare og spesifikke instruksjoner

### Taktikk 1 - Bruk skilletegn for å tydelig indikere distinkte deler av input
Ved å bruke skilletegn kan du tydelig indikere distinkte deler av input. Skilletegn kan være: ```, """, < >, `<tag> </tag>`, `:`.

In [ ]:
text = """
For a model to work well, just be really clear \ 
about what you want. Give specific instructions \ 
so the model knows exactly what to do. This helps \ 
get the right answers and avoids mistakes. Remember, \ 
a short prompt isn't always a clear one. Sometimes, \ 
it's better to use longer prompts to give more \ 
info to the model. This usually leads to better \ 
and more relevant results.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.

```{text}```
"""

response = get_completion(prompt)
pprint(response)

### Taktikk 2 - Be om en strukturert output
Ved å be om en strukturert output kan du få resultater som er enklere å analysere og bruke i applikasjoner. Dette er spesielt nyttig når du vil bruke output i en videre prosess. Eksempler på strukturert output er JSON og HTML.

In [ ]:
prompt = f"""
Generate a list of three fictional banking product names along with their developers and categories. 
Provide them in JSON format with the following keys: product_id, name, developer, category.
"""

response = get_completion(prompt)
print(response)

### Taktikk 3 - Be modellen sjekke om betingelser er oppfylt
Modellen kan bli bedt om å gjøre resonnementsteg. For eksempel kan den sjekke en betingelse, og utføre en handling avhengig av resultatet.

In [ ]:
# Example asking the model to check a condition - part 1 - condition met
text_1 = f"""
Understanding Insurance Basics is Simple! First, identify the type of coverage you need. 
While doing that, gather relevant information about your assets and risks. 
Next, choose a reputable insurance provider and select a suitable policy. 
Once you've made your decision, go ahead and complete the application process. 
Wait for the policy to be approved. After approval, review the terms and conditions carefully. 
If everything looks good, make the required payments to activate your coverage. 
If needed, consult with an insurance agent for clarification. 
And there you have it! You've successfully secured insurance coverage for your peace of mind.
"""

# Prompt with a conditional instruction + text input
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print(response)


In [ ]:
# Example asking the model to check a condition - part 2 - condition not met
text_2 = f"""
The stock market is buzzing with activity today, and financial analysts are deep in discussions. 
It's an intriguing day to explore the dynamics of the financial world. 
The charts are fluctuating, and economic indicators are being closely monitored. 
Traders are making strategic moves, and investors are assessing their portfolios. 
Some are engaged in intense conversations about market trends, while others are researching potential investment opportunities. 
It's an ideal day to delve into the intricacies of finance and witness the ebb and flow of economic activities.
"""

# Prompt with a conditional instruction + text input
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)
print(response)

### Taktikk 4 - Gi eksempler
Ved å gi modellen eksempler på ønsket output kan den lære seg å generalisere til lignende tilfeller.

In [ ]:
# Providing a sample conversation
prompt = f"""
Your task is to answer in a consistent style.

<client>: Enlighten me about financial planning.

<financial_advisor>: The fortune that grows most steadily \
emanates from disciplined savings; the \
most prosperous investment portfolio begins with a single asset; \
the most secure financial future starts with a well-laid plan.

<client>: Enlighten me about risk management.
"""

response = get_completion(prompt)
pprint(response)

## 1.2 Prinsipp 2: Gi modellen tid til å "tenke"

<center><img src="resources/Thinking-Fast-and-Slow.jpg" alt="Prompting principle 2: time to think"/>

### Taktikk 1 - Spesifiser trinnene som kreves for å fullføre oppgaven
Ved å spesifisere trinnene kan modellen lære å utføre oppgaven trinn for trinn i stedet for å forhaste seg til en konklusjon.

In [ ]:
# Example using complex (and slightly pointless) step by step instructions

text = f"""
In a bustling city, siblings Alex and Emma embarked on \
a mission to secure insurance coverage for their \
family. As they navigated the policy options, discussing earnestly, uncertainty \
struck—Alex faced unexpected challenges understanding the terms \
of coverage, and Emma encountered confusing jargon. \
Though slightly perplexed, the duo sought assistance from \
knowledgeable advisors. Despite the confusion, \
their determined efforts prevailed, and they \
left the insurance agency with a sense of security and understanding.
"""

# step-by-step instructions:
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print(response)

### Taktikk 2 - Instruer modellen til å finne sin egen løsning før den konkluderer

Ved å instruere modellen til å utarbeide sin egen løsning før den konkluderer, kan vi unngå dette problemet. I dette eksemplet kan vi se modellen bli "fristet" av en enkel løsning.

In [ ]:
# Jumping to wrong conclusion
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.

Question:
```
I'm starting a financial consultancy and need assistance
working out the budget.
- Office space rental costs $50 / square foot 
- I can purchase computer equipment for $200 / square foot
- I negotiated an annual contract for IT support that will cost
me a flat $80k per year, and an additional $15 / square
foot
What is the total budget for the first year of operations
as a function of the number of square feet?
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Office cost: 50x
2. Solar panel cost: 200x
3. Maintenance cost: 80,000 + 150x
Total cost: 50x + 200x + 80,000 + 150x = 400x + 80,000

"""
response = get_completion(prompt)
pprint(response)

Merk at elevens løsning faktisk ikke er riktig. Vi kan fikse dette ved å instruere modellen til å utarbeide sin egen løsning først.

In [ ]:
# Thinking things through, and getting it right
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm starting a financial consultancy and need assistance
working out the budget.
- Office space rental costs $50 / square foot 
- I can purchase computer equipment for $200 / square foot
- I negotiated an annual contract for IT support that will cost
me a flat $80k per year, and an additional $15 / square
foot
What is the total budget for the first year of operations
as a function of the number of square feet?
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Office cost: 50x
2. Solar panel cost: 200x
3. Maintenance cost: 80,000 + 150x
Total cost: 50x + 200x + 80,000 + 150x = 400x + 80,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

# 2. Tre viktige prompt engineering-strategier
Eksemplene ovenfor illustrerer hvordan en kan få en LLM-modell til å utføre vanlige oppgaver. Grovt sett kan de klassifiseres innenfor de definerte prompt engineering-strategiene Zero-Shot, Few-Shot og Chain-of-Thoughts.

## 2.1 Zero-Shot-prompting

LLM-er, som ChatGPT, er utviklet til å følge instruksjoner og er trent på store datamengder. Derfor er de i stand til å utføre noen oppgaver "zero-shot". Det betyr at modellen kan utføre en oppgave uten finjustering eller trening. Her er ett av eksemplene vi brukte tidligere som er "zero-shot".   

In [ ]:
prompt = f"""
Generate a list of three fictional banking product names along with their developers and categories. 
Provide them in JSON format with the following keys: product_id, name, developer, category.
"""

Merk at ovenfor ga vi ikke modellen noen eksempler på bankprodukter eller JSON-format. Dette er "zero-shot"-funksjonen.


Når zero-shot ikke fungerer anbefales det å gi demonstrasjoner eller eksempler og det fører oss til "few-shot"-prompting.

## 2.2 Few-Shot-prompting

Mens LLM'er demonstrerer bemerkelsesverdige "zero-shot"-evner kommer de fortsatt til kort ved mer komplekse oppgaver. Few-shot prompting kan brukes for å muliggjøre læring i kontekst, og vi kan dermed gi eksempler for å styre modellen til bedre ytelse. Demonstrasjonene fungerer som kondisjonering for påfølgende eksempler der vi ønsker at modellen skal generere en respons.

La oss si at du er en basketballprodusent og ønsker å lage en sentimentanalyse basert på kundeanmeldelser. Da kan du bruke "Few-Shot"-prompting for å styre modellen til bedre ytelse. Her er et eksempel på hvordan du kan gjøre det:

In [ ]:
# Zero-shot attempt
prompt = f"""
Determine the sentiment of this sentence.
Sentence: This basketball has a lot of weight.
"""

response = get_completion(prompt)
print(response)

Selv om modellen klassifiserer anmeldelsen som nøytral kan en basketball med mye vekt være et defekt produkt. Derfor kan vi bruke "Few-Shot"-prompting for å styre modellen til bedre ytelse.

In [ ]:
# Few-shot attempt
prompt = f"""
Sentence: This basketball is easy to carry.

Answer: The sentiment of this sentence is positive.

Determine the sentiment of the sentence.

Sentence: This basketball has a lot of weight.

Answer:
"""

response = get_completion(prompt)
print(response)

Denne gangen klassifiserer modellen anmeldelsen som 'negativ' ettersom den bruker det angitte eksemplet som referanse.

Selv om "Few-Shot"-prompting er en kraftig teknikk har den noen begrensninger - som vist her:

In [ ]:
# Unsuccessful few-shot attempt at average calculation
prompt = f"""
You are an AI assistant capable of performing complex  numerical computations.

Question 1:
Find the average of the numbers. Here are the numbers: 1, 2, 3.

Answer 1:
The average is 2.

Question 2:
Find the average of the numbers. Here are the numbers: 5, 10, 15, 7.

Answer 2:
The average is 9.25.

Question 3:
Find the average of the numbers. Here are the numbers: 63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145.
"""

response = get_completion(prompt)
print(response)


In [ ]:
# Checking the calculation: 
x = [63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145]
sum(x)/len(x)

Som vi kan se er ikke dette svaret riktig og det er behov for mer avansert prompt engineering.

## 2.3 Chain-of-thoughts-prompting

"Chain of thought"-prompting muliggjør kompleks resonnering gjennom mellomliggende resonnementtrinn. Det kan kombineres med "Few-Shot"-prompting for å få bedre resultater på mer komplekse oppgaver som krever resonnering.

In [ ]:
# Average calculation with combined few-shot and chain-of-thoughts prompting
prompt = f"""
You are an AI assistant capable of performing complex  numerical computations.

Question 1:
Find the average of the numbers. Here are the numbers: 1, 2, 3.

Answer 1:
We start by finding the sum of the numbers. 1+2=3, 3+3=6. The sum of the numbers is 6. Then we divide the sum by the number of numbers. 6/3=2. The average is 2.

Question 2:
Find the average of the numbers. Here are the numbers: 5, 10, 15, 7.

Answer 2:
We start by finding the sum of the numbers. 5+10=15, 15+15=30, 30+7=37. The sum of the numbers is 37. Then we divide the sum by the number of numbers. 37/4=9.25. The average is 9.25.

Question 3:
Find the average of the numbers. Here are the numbers: 63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145.
"""

response = get_completion(prompt)
pprint(response)

Som vi kan se er modellen i stand til å resonnere rundt spørsmålet og gi et riktig svar når det gis mer detaljerte instruksjoner om hvordan problemet skal løses.

# 3. Prompteksempler for vanlige LLM-oppgaver

1. Er det tydelig forskjell mellom modellene? Hvis ja, hva er forskjellen?
2. Hva kan være utfordrende ved bruk av LLM-er og arbeid med prompts?
3. Refleksjoner rundt arbeid med prompts fra et utviklerperspektiv.
    (Utivkle basert på en eldre/dårligere modell og når jeg har en fungerende flyt, bytter jeg til en nyere/bedre modell.)
    (Versjonering av prompts)

## 3.1 Oppsummering
### Oppsummering med ord, setninger eller tegngrenser

In [ ]:
# Sample text for summary examples
prod_review = """
Acquired this 'FjordGuard Assurance' policy for my family's financial security, and they find it reassuring. 
The coverage is comprehensive, and the policy terms are transparent. The customer service has a welcoming approach, providing a sense of trust. 
However, I feel the premium is a bit high for the coverage offered. 
There might be other insurance options with more extensive benefits at a similar cost. 
Surprisingly, the policy documentation arrived a day earlier than expected, allowing me to review it thoroughly before presenting it to my family.
"""

In [ ]:
# Straightforward summary
prompt = f"""
Your task is to generate a short summary of a product \
review. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

### Oppsummer med et spesifikt fokus

Vi kan be om et sammendrag med spesifikt fokus, og få et annet svar.

In [ ]:
# Summary with specific focus
 
prompt = f"""
Your task is to generate a short summary of a product \
review to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

Som vi kan se inkluderer oppsummeringene emner som ikke er relatert til fokustemaet. Endring av ordlyden i promptet kan noen ganger gi bedre resultater.

In [ ]:
# Asking for information extract instead of a summary
prompt = f"""
Your task is to extract relevant information from \ 
a product review to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

## 3.2 Inferering og kategorisering

Et standard NLP-problem er sentimentanalyse og vi kan bruke LLM-er til å utføre denne oppgaven.

In [ ]:
# Sample review text
review = """
Needed a reliable financial advisor for my investments, and NordWealth stood out with additional services and reasonable fees. After purchasing their comprehensive financial planning package, the onboarding process was swift. The communication channel broke during the initial consultation, and the company promptly arranged for a follow-up. The financial plan was delivered within a few days, and the advisor explained it thoroughly. Setting up the investment portfolio was a breeze. I encountered a minor issue, so I contacted their support, and they promptly resolved the matter. NordWealth appears to be an excellent financial service provider that prioritizes its clients and their financial goals!
"""

In [ ]:
# Review sentiment determined using LLM
prompt = f"""
What is the sentiment of the following review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

En annen kraftig egenskap av LLM-er er å trekke ut informasjon. Et eksempel der dette er nyttig er når du ønsker å trekke ut informasjon om et produkt eller tjeneste fra anmeldelser.

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{review}'''
"""

response = get_completion(prompt)
print(response)

LLM-er er også nyttige til å kategorisere og merke tekst i henhold til innhold.

In [ ]:
story = """
In a recent governmental survey examining employee satisfaction within the Norwegian financial landscape, participants were tasked with evaluating their contentment levels within their respective banking institutions. The findings illuminated that NorthHarbor Financial Alliance emerged as the leading institution, securing a commendable satisfaction rating of 85%.

One employee from NorthHarbor Financial Alliance, Isabella Nordstrøm, shared her perspective on the results, stating, "The recognition of NorthHarbor doesn't come as a surprise. It's a remarkable workplace with a collaborative culture and ample growth opportunities. I take pride in contributing to such a distinguished financial institution."

The positive outcomes were warmly embraced by NorthHarbor Financial Alliance's leadership, with CEO Anders Olsen expressing, "We are elated to witness such robust levels of satisfaction among our dedicated team. Our collective efforts consistently aim for excellence, and it's genuinely fulfilling to see our workforce's contributions acknowledged."

Conversely, the survey spotlighted that Nordic Finance Collective exhibited the lowest satisfaction rating, with only 38% of employees expressing contentment in their roles. In response, the government has committed to addressing the concerns raised in the survey and is actively working toward enhancing overall job satisfaction throughout the financial sector.
"""

In [ ]:
# Extracting topics from text
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""

response = get_completion(prompt)
print(response)

## 3.3 Transformering

LLM-er er trent på mange kilder på forskjellige språk - dette gir modellen muligheten til å gjøre oversettelser.

In [ ]:
# Effortless translation
prompt = f"""
Translate the following English text to Spanish and English pirate: \ 
```Hello, I would like to purchase an insurance package.```
"""

response = get_completion(prompt)
pprint(response)

Tekst kan variere i tone og bør være basert på den tiltenkte målgruppen. LLM-er er svært effektive når det gjelder å endre tonen i en gitt tekst.

In [ ]:
# Changing tone of voice
prompt = f"""
Translate the following from slang to a business letter: 
'Dude! Joey here. Check out the specs for this savings account.'
"""

response = get_completion(prompt)
pprint(response)

## 3.4 Kundeanmeldelser

Under laster vi inn en dataframe med kundeanmeldelsene vi brukte tidligere.

Oppgaver:

1. Lag en sentimentanalyse og/eller en rangering av anmeldelsene.
2. Er det et gjengående tema i de positive/negative anmeldelsene?

In [ ]:
import pandas as pd

df_text = pd.read_csv('../Del_2_Litt_om_OpenAI_APIer/data/text_samples_bytt.csv', header=0, sep=';')

df_text.head()